<a href="https://colab.research.google.com/github/EmilyKan/pycs4001/blob/main/titanic_rf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm


test_data=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/test.csv")
train_data=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/train.csv")
gender_data=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/gender_submission.csv")
test_ids = test_data["PassengerId"]

def clean(data):
    data = data.drop(["PassengerId", "Name", "Ticket", "Cabin", "Embarked"], axis=1)

    cols = ["SibSp", "Parch", "Fare", "Age"]
    for col in cols:
        data[col].fillna(data[col].mean(), inplace=True)
    return data



train_data = clean(train_data)
test_data = clean(test_data)

train_data.head(5)

X = train_data.iloc[:, 1:]
Y = train_data.iloc[:, 0]

print(f"train data shape: {X.shape}")
print(f"test data shape: {test_data.shape}")

dummy_x = pd.get_dummies(X)
print(dummy_x.shape)
dummy_x


from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
sc_x = scaler.fit_transform(dummy_x)

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

depth = []
for i in tqdm(range(2,50,1)):
  rf = RandomForestClassifier(max_depth=i)
  scores = cross_val_score(rf, sc_x, Y, cv=5)
  depth.append(scores.mean())

final_depth = np.argmax(np.array(depth)) + 2
print(final_depth)

estimators = []
for i in tqdm(range(50,200,1)):
  rf = RandomForestClassifier(n_estimators=i, max_depth=final_depth)
  scores = cross_val_score(rf, sc_x, Y, cv=5)
  estimators.append(scores.mean())


final_estimators = np.argmax(np.array(estimators)) + 50
print(final_estimators)

final_rf = RandomForestClassifier(n_estimators=final_estimators, max_depth=final_depth)
final_rf.fit(sc_x, Y)

estimators = []
for i in tqdm(range(50,200,1)):
  rf = RandomForestClassifier(n_estimators=i, max_depth=final_depth)
  scores = cross_val_score(rf, sc_x, Y, cv=5)
  estimators.append(scores.mean())





dummy_test_data = pd.get_dummies(test_data)
dummy_test_data

sc_test_data_x = scaler.transform(dummy_test_data)

submission = final_rf.predict(sc_test_data_x)

print(submission)

submission_data = pd.DataFrame({"PassengerId": test_ids.values, "Survived": submission })
submission_data.to_csv("submission.csv", index=False)


train data shape: (891, 6)
test data shape: (418, 6)
(891, 7)


100%|██████████| 28/28 [00:24<00:00,  1.16it/s]


10


 77%|███████▋  | 115/150 [01:49<00:49,  1.41s/it]